<a href="https://colab.research.google.com/github/wuchenyu38/18ma573chenyuwu/blob/master/Heston_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/wuchenyu38/18ma573chenyuwu.git

Cloning into '18ma573chenyuwu'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 312 (delta 35), reused 0 (delta 0), pack-reused 203
Receiving objects: 100% (312/312), 420.11 KiB | 10.77 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [0]:
cd 18ma573chenyuwu/src

/content/18ma573chenyuwu/src


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as int
import BSM as bs

Based on the fourier transformation:
$$
C=S_0I_1-Ke^{-rT}I_2
$$
$$
I_1=\frac{1}{2}+\frac{1}{\pi}\int_0^{\infty}Re(\frac{e^{-iuln(K)}\phi(u-i)}{iu\phi(-i)})du
$$$$
I_2=\frac{1}{2}+\frac{1}{\pi}\int_0^{\infty}Re(\frac{e^{-iuln(K)}\phi(u)}{iu})du
$$

$\kappa = 1.2$; $\theta = .04$; $\xi = .3$; $\rho = .5$; $\tau = T-t$

From Ali Hirsa's Book:
$$
\Phi(u)=E(e^{iulnS_t})=\frac{exp(iulnS_0+iu(r-q)t+\frac{\kappa \theta t(\kappa-i\rho\sigma u)}{\sigma^2})}{(cosh\frac{\gamma t}{2}+\frac{\kappa-i\rho\sigma u}{\gamma}sinh\frac{\gamma t}{2})^{\frac{2\kappa \theta}{\sigma ^2}}}exp(\frac{-(u^2+iu)v_0}{\gamma coth\frac{\gamma t}{2}+\kappa-i\rho \sigma u})
$$

$$
\beta = \kappa - i \xi \rho \sigma
\\
\gamma =\sqrt{(\kappa - i \xi \rho \sigma)^2+\sigma ^2 (\xi ^2+i \xi)}
$$

$$
E_t(e^{iux_T})=\frac{exp(iulnS_0+iu(r-q)\tau+\kappa \theta \tau \frac{\beta}{\sigma ^2})exp(\frac{-(u^2+iu)v_0}{\gamma coth\frac{\gamma \tau}{2}+\beta})}{(cosh\frac{\gamma \tau}{2}+\frac{\beta}{\gamma}sinh\frac{\gamma \tau}{2})^{\frac{2\kappa \theta}{\sigma ^2}}}
$$

$\gamma = \sqrt{(\kappa - i\xi \rho u)^2+\sigma^2 (u^2+ iu)}$
</br>
$\beta = \kappa-i \xi \rho u$
</br>
$\xi $ is the volatiliy of volatility which is equal to $\sigma$

In [0]:
kappa = 1.2; theta = .04; xi = .3; rho = .5;

In [0]:
T = 1; s0=100.; v0=.04; r=.05; otype=1; market_price = 15; K =100; 

In [0]:
def Heston(kappa,theta, xi, rho, s0, v0, T, t, r, q, u):
  tau= T-t
  beta=kappa-1j*xi*rho*u
  gamma=np.sqrt((kappa-1j*xi*rho*u)**2+(xi**2)*(u**2+1j*u))
  p1=np.exp(1j*u*np.log(s0)+1j*u*(r-q)*tau+kappa*theta*tau*(beta/xi**2))
  p2=np.exp(-(u**2+1j*u)*v0/(gamma*(np.cosh(gamma*T/2)/np.sinh(gamma*T/2))+beta))
  p3=(np.cosh(gamma*tau/2)+(beta/gamma)*np.sinh(gamma*tau/2))**(2*kappa*theta/xi**2)
  return p1*p2/p3

In [0]:
def Re1(kappa, theta, xi, rho, s0, v0, T, t, r, q, u):
  return (np.exp(-1j*u*np.log(K/s0)*u)*Heston(kappa, theta,xi, rho, s0, v0, T, t, r, q, u-1j)/(1j*u*Heston(kappa, theta, xi, rho, s0, v0, T, t, r, q, -1j))).real

In [0]:
def Re2(kappa, theta, xi, rho, s0, v0, T, t, r, q, u):
  return (np.exp(-1j*np.log(K/s0)*u)*Heston(kappa, theta, xi, rho, s0, v0, T, t, r, q, u)/(1j*u)).real

In [0]:
I1=1/2+(1/np.pi)*int.quad(lambda u: Re1(kappa, theta, xi, rho, s0, v0, T, 0, r, 0, u),0,5000)[0]
I2=1/2+(1/np.pi)*int.quad(lambda u: Re2(kappa, theta, xi, rho, s0, v0, T, 0, r, 0, u),0,5000)[0]

C=(s0*I1)-(K*(np.exp(-1*r*T))*I2)
print("Price = " + str(C))

Price = 4.877057551096641


Problem: Integration breaks after 5000